# Hierarchical Forecast Reconciliation with Uncertainty Quantification

This notebook explores the M5 dataset and demonstrates the hierarchical forecasting approach with uncertainty quantification.

## Overview

This project implements a novel ensemble approach that combines:
- **Statistical models**: ETS and ARIMA for baseline forecasting
- **Deep learning models**: Temporal Fusion Transformer and N-BEATS
- **Hierarchical reconciliation**: Probabilistic reconciliation that preserves prediction intervals
- **Uncertainty quantification**: Coherent prediction intervals across all hierarchy levels

## Setup and Imports

In [ ]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Statistical analysis
from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Project imports
sys.path.append('../src')
from hierarchical_forecast_reconciliation_with_uncertainty_quantification.utils.config import (
    load_config, set_random_seeds, setup_logging
)
from hierarchical_forecast_reconciliation_with_uncertainty_quantification.data.loader import (
    M5DataLoader, HierarchicalDataBuilder
)
from hierarchical_forecast_reconciliation_with_uncertainty_quantification.data.preprocessing import (
    M5Preprocessor, HierarchyBuilder
)
from hierarchical_forecast_reconciliation_with_uncertainty_quantification.models.model import (
    StatisticalForecaster, HierarchicalEnsembleForecaster
)
from hierarchical_forecast_reconciliation_with_uncertainty_quantification.evaluation.metrics import (
    HierarchicalMetrics
)

# Configure environment
warnings.filterwarnings('ignore')
set_random_seeds(42)
setup_logging(level='INFO', console=True)

# Plotting configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

print("Setup complete!")

## Configuration and Data Loading

In [ ]:
# Load configuration
config = load_config('../configs/default.yaml')

# Update data path for demo (you would set this to your actual M5 data path)
# config['data']['path'] = '/path/to/m5/data'

print("Configuration loaded:")
print(f"- Aggregation levels: {len(config['data']['aggregation_levels'])}")
print(f"- Statistical models: {list(config['models']['statistical'].keys())}")
print(f"- Deep learning models: {list(config['models']['deep_learning'].keys())}")
print(f"- Ensemble weights: {config['ensemble']['weights']}")

## Generate Synthetic M5-like Data for Demonstration

Since the actual M5 dataset is large, we'll generate synthetic data that follows similar patterns.

In [ ]:
def generate_synthetic_m5_data(n_days=1969, n_items=50, seasonal_strength=0.3, trend_strength=0.1):
    """
    Generate synthetic M5-like hierarchical time series data.
    """
    np.random.seed(42)
    
    # Date range
    dates = pd.date_range('2016-01-01', periods=n_days, freq='D')
    
    # Hierarchy structure
    states = ['CA', 'TX', 'WI']
    categories = ['FOODS', 'HOBBIES', 'HOUSEHOLD']
    stores_per_state = {'CA': 4, 'TX': 3, 'WI': 3}
    items_per_category = n_items // len(categories)
    
    data_rows = []
    
    for state in states:
        for store_idx in range(stores_per_state[state]):
            store_id = f"{state}_{store_idx + 1}"
            
            for cat in categories:
                for item_idx in range(items_per_category):
                    item_id = f"{cat}_{item_idx + 1:03d}"
                    series_id = f"{item_id}_{store_id}_validation"
                    
                    # Generate base pattern
                    t = np.arange(n_days)
                    
                    # Trend component
                    trend = trend_strength * t / 365  # Yearly trend
                    
                    # Seasonal components
                    weekly_seasonal = seasonal_strength * np.sin(2 * np.pi * t / 7)
                    yearly_seasonal = seasonal_strength * 0.5 * np.sin(2 * np.pi * t / 365.25)
                    
                    # Base level (varies by category)
                    base_levels = {'FOODS': 5, 'HOBBIES': 2, 'HOUSEHOLD': 3}
                    base_level = base_levels[cat]
                    
                    # Store effect
                    store_effect = np.random.normal(1, 0.2)
                    
                    # Noise
                    noise = np.random.normal(0, 0.5, n_days)
                    
                    # Combine components
                    sales = np.maximum(0, 
                        base_level * store_effect + trend + weekly_seasonal + yearly_seasonal + noise
                    )
                    
                    # Add some intermittency
                    intermittency = np.random.binomial(1, 0.85, n_days)  # 15% zero sales
                    sales = sales * intermittency
                    
                    for day, (date, sale) in enumerate(zip(dates, sales)):
                        data_rows.append({
                            'id': series_id,
                            'item_id': item_id,
                            'dept_id': cat,
                            'cat_id': cat,
                            'store_id': store_id,
                            'state_id': state,
                            'date': date,
                            'sales': sale,
                            'd': f'd_{day + 1}'
                        })
    
    return pd.DataFrame(data_rows)

# Generate synthetic data
print("Generating synthetic M5-like data...")
synthetic_data = generate_synthetic_m5_data()

print(f"Generated data shape: {synthetic_data.shape}")
print(f"Date range: {synthetic_data['date'].min()} to {synthetic_data['date'].max()}")
print(f"Number of series: {synthetic_data['id'].nunique()}")
print(f"Hierarchy levels:")
print(f"  - States: {synthetic_data['state_id'].nunique()}")
print(f"  - Stores: {synthetic_data['store_id'].nunique()}")
print(f"  - Categories: {synthetic_data['cat_id'].nunique()}")
print(f"  - Items: {synthetic_data['item_id'].nunique()}")

## Data Exploration and Visualization

### Overall Sales Trends

In [ ]:
# Aggregate sales by date
daily_total = synthetic_data.groupby('date')['sales'].sum().reset_index()

# Create interactive plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=daily_total['date'],
    y=daily_total['sales'],
    mode='lines',
    name='Total Sales',
    line=dict(color='blue', width=1)
))

# Add 30-day moving average
daily_total['ma_30'] = daily_total['sales'].rolling(window=30).mean()
fig.add_trace(go.Scatter(
    x=daily_total['date'],
    y=daily_total['ma_30'],
    mode='lines',
    name='30-day MA',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='Total Daily Sales Over Time',
    xaxis_title='Date',
    yaxis_title='Sales Units',
    height=500,
    hovermode='x unified'
)

fig.show()

# Summary statistics
print("\nDaily Sales Statistics:")
print(daily_total['sales'].describe())

### Sales by Hierarchy Levels

In [ ]:
# Sales by state
state_sales = synthetic_data.groupby(['date', 'state_id'])['sales'].sum().reset_index()

fig = px.line(state_sales, x='date', y='sales', color='state_id',
              title='Sales by State Over Time',
              labels={'sales': 'Sales Units', 'date': 'Date'})
fig.update_layout(height=400)
fig.show()

# Sales by category
category_sales = synthetic_data.groupby(['date', 'cat_id'])['sales'].sum().reset_index()

fig = px.line(category_sales, x='date', y='sales', color='cat_id',
              title='Sales by Category Over Time',
              labels={'sales': 'Sales Units', 'date': 'Date'})
fig.update_layout(height=400)
fig.show()

### Seasonal Patterns

In [ ]:
# Add time features
synthetic_data['weekday'] = synthetic_data['date'].dt.day_name()
synthetic_data['month'] = synthetic_data['date'].dt.month
synthetic_data['week_of_year'] = synthetic_data['date'].dt.isocalendar().week

# Weekly patterns
weekly_pattern = synthetic_data.groupby('weekday')['sales'].mean().reset_index()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekly_pattern['weekday'] = pd.Categorical(weekly_pattern['weekday'], categories=weekday_order, ordered=True)
weekly_pattern = weekly_pattern.sort_values('weekday')

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Weekly pattern
axes[0, 0].bar(weekly_pattern['weekday'], weekly_pattern['sales'], color='skyblue')
axes[0, 0].set_title('Average Sales by Day of Week')
axes[0, 0].set_ylabel('Sales Units')
axes[0, 0].tick_params(axis='x', rotation=45)

# Monthly pattern
monthly_pattern = synthetic_data.groupby('month')['sales'].mean().reset_index()
axes[0, 1].plot(monthly_pattern['month'], monthly_pattern['sales'], marker='o', color='orange')
axes[0, 1].set_title('Average Sales by Month')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Sales Units')
axes[0, 1].grid(True, alpha=0.3)

# Sales distribution
axes[1, 0].hist(synthetic_data[synthetic_data['sales'] > 0]['sales'], bins=50, alpha=0.7, color='green')
axes[1, 0].set_title('Sales Distribution (Non-zero)')
axes[1, 0].set_xlabel('Sales Units')
axes[1, 0].set_ylabel('Frequency')

# Zero sales percentage
zero_sales_pct = synthetic_data.groupby('id').apply(
    lambda x: (x['sales'] == 0).mean() * 100
).reset_index(name='zero_pct')

axes[1, 1].hist(zero_sales_pct['zero_pct'], bins=20, alpha=0.7, color='coral')
axes[1, 1].set_title('Distribution of Zero Sales Percentage by Series')
axes[1, 1].set_xlabel('Zero Sales Percentage')
axes[1, 1].set_ylabel('Number of Series')

plt.tight_layout()
plt.show()

print(f"\nOverall zero sales rate: {(synthetic_data['sales'] == 0).mean():.1%}")
print(f"Average zero sales rate per series: {zero_sales_pct['zero_pct'].mean():.1f}%")

### Hierarchy Structure Analysis

In [ ]:
# Build hierarchical aggregations
hierarchy_builder = HierarchicalDataBuilder(config['data']['aggregation_levels'])
hierarchy_data = hierarchy_builder.build_hierarchy(synthetic_data)

# Analyze hierarchy structure
matrix_builder = HierarchyBuilder(config['data']['aggregation_levels'])
structure_info = matrix_builder.get_hierarchy_structure(hierarchy_data)

print("Hierarchy Structure:")
print("-" * 50)
total_series = 0
for level, info in structure_info.items():
    print(f"{level:15s}: {info['n_series']:4d} series, {info['total_observations']:7d} observations")
    total_series += info['n_series']

print("-" * 50)
print(f"{'Total':15s}: {total_series:4d} series across all levels")

# Visualize hierarchy structure
levels = list(structure_info.keys())
n_series = [structure_info[level]['n_series'] for level in levels]

fig, ax = plt.subplots(1, 1, figsize=(12, 6))
bars = ax.bar(levels, n_series, color='lightblue', edgecolor='navy', alpha=0.7)
ax.set_title('Number of Series by Hierarchy Level', fontsize=14, fontweight='bold')
ax.set_ylabel('Number of Series')
ax.set_xlabel('Hierarchy Level')

# Add value labels on bars
for bar, value in zip(bars, n_series):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(n_series) * 0.01,
            str(value), ha='center', va='bottom', fontweight='bold')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Data Preprocessing Pipeline

In [ ]:
# Initialize preprocessor
preprocessor = M5Preprocessor(
    scaling_method="standard",
    handle_zeros="log1p",
    outlier_threshold=3.0
)

# Preprocess data
print("Preprocessing data...")
processed_data = preprocessor.fit_transform(synthetic_data)

print(f"Original data shape: {synthetic_data.shape}")
print(f"Processed data shape: {processed_data.shape}")
print(f"Added features: {processed_data.shape[1] - synthetic_data.shape[1]}")

# Show new features
new_features = [col for col in processed_data.columns if col not in synthetic_data.columns]
print(f"\nNew features added: {new_features[:10]}...")  # Show first 10

# Create train/validation/test splits
train_data, val_data, test_data = preprocessor.create_train_test_split(
    processed_data,
    train_days=config['data']['train_days'],
    validation_days=config['data']['validation_days'],
    test_days=config['data']['test_days']
)

print(f"\nData splits:")
print(f"  Train: {train_data.shape[0]:,} observations")
print(f"  Validation: {val_data.shape[0]:,} observations")
print(f"  Test: {test_data.shape[0]:,} observations")

## Model Demonstration

### Statistical Model Example

In [ ]:
# Demonstrate statistical forecaster with a subset of data
# Select a few representative series for demonstration
demo_series = train_data['id'].unique()[:5]
demo_train = train_data[train_data['id'].isin(demo_series)]

print(f"Training statistical model on {len(demo_series)} series...")

# Initialize and fit statistical forecaster
stat_forecaster = StatisticalForecaster(
    method="ets",
    ets_config=config['models']['statistical']['ets']
)

try:
    stat_forecaster.fit(demo_train)
    
    # Generate predictions
    horizon = 28
    stat_predictions = stat_forecaster.predict(
        horizon=horizon,
        return_intervals=True,
        confidence_levels=[0.1, 0.05]
    )
    
    print("Statistical model trained successfully!")
    print(f"Generated forecasts for {len(stat_predictions['forecasts'])} series")
    
    # Visualize predictions for first series
    series_id = list(stat_predictions['forecasts'].keys())[0]
    
    # Get historical data for this series
    hist_data = demo_train[demo_train['id'] == series_id].sort_values('date')
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    # Plot historical data (last 100 points)
    hist_plot = hist_data.tail(100)
    ax.plot(range(len(hist_plot)), hist_plot['sales'], 'b-', label='Historical', alpha=0.8)
    
    # Plot forecast
    forecast_start = len(hist_plot)
    forecast_range = range(forecast_start, forecast_start + horizon)
    
    forecast = stat_predictions['forecasts'][series_id]
    ax.plot(forecast_range, forecast, 'r-', label='Forecast', linewidth=2)
    
    # Plot confidence intervals
    if 'lower_90' in stat_predictions:
        lower_90 = stat_predictions['lower_90'][series_id]
        upper_90 = stat_predictions['upper_90'][series_id]
        ax.fill_between(forecast_range, lower_90, upper_90, alpha=0.3, color='red', label='90% CI')
    
    ax.axvline(x=forecast_start, color='black', linestyle='--', alpha=0.7, label='Forecast Start')
    ax.set_title(f'Statistical Model Forecast: {series_id[:30]}...')
    ax.set_xlabel('Time')
    ax.set_ylabel('Sales')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error in statistical modeling: {e}")
    print("This is expected in a demo environment without full dependencies.")

### Evaluation Metrics Demonstration

In [ ]:
# Demonstrate evaluation metrics with synthetic predictions
evaluator = HierarchicalMetrics()

# Generate synthetic predictions and actuals for demonstration
np.random.seed(42)
demo_horizon = 28

demo_predictions = {}
demo_actuals = {}
demo_intervals = {'lower_90': {}, 'upper_90': {}, 'lower_95': {}, 'upper_95': {}}

for series_id in demo_series:
    # Generate realistic predictions (with some error)
    true_values = np.random.uniform(2, 8, demo_horizon)
    predictions = true_values + np.random.normal(0, 0.5, demo_horizon)
    
    demo_predictions[series_id] = predictions
    demo_actuals[series_id] = true_values
    
    # Generate prediction intervals
    demo_intervals['lower_90'][series_id] = predictions - 1.645 * 0.5
    demo_intervals['upper_90'][series_id] = predictions + 1.645 * 0.5
    demo_intervals['lower_95'][series_id] = predictions - 1.96 * 0.5
    demo_intervals['upper_95'][series_id] = predictions + 1.96 * 0.5

# Compute metrics
metrics = evaluator.compute_all_metrics(
    predictions=demo_predictions,
    actuals=demo_actuals,
    intervals=demo_intervals,
    confidence_levels=[0.1, 0.05]
)

print("Evaluation Metrics (Synthetic Example):")
print("=" * 50)
for metric_name, value in metrics.items():
    print(f"{metric_name:25s}: {value:.4f}")

# Generate performance report
target_metrics = {
    "WRMSSE": 0.52,
    "coverage_90": 0.88,
    "coverage_95": 0.94
}

report = evaluator.create_performance_report(metrics, target_metrics)
print("\n" + report)

### Hierarchical Reconciliation Visualization

In [ ]:
# Demonstrate the concept of hierarchical reconciliation
from hierarchical_forecast_reconciliation_with_uncertainty_quantification.models.model import ProbabilisticReconciler

# Build aggregation matrix
aggregation_matrix = matrix_builder.build_aggregation_matrix(hierarchy_data)

print(f"Aggregation matrix shape: {aggregation_matrix.shape}")
print(f"Matrix density: {aggregation_matrix.nnz / (aggregation_matrix.shape[0] * aggregation_matrix.shape[1]):.3f}")

# Visualize a small portion of the aggregation matrix
# Show first 20 rows and columns as dense matrix
matrix_sample = aggregation_matrix[:20, :20].toarray()

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
im = ax.imshow(matrix_sample, cmap='Blues', aspect='auto')
ax.set_title('Aggregation Matrix Sample (First 20x20)', fontsize=14)
ax.set_xlabel('Bottom-level Series')
ax.set_ylabel('Aggregated Series')

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Aggregation Weight')

plt.tight_layout()
plt.show()

print("\nAggregation Matrix Properties:")
print(f"- Each row represents an aggregated series")
print(f"- Each column represents a bottom-level series")
print(f"- Non-zero entries indicate which bottom series contribute to each aggregate")
print(f"- The matrix enforces hierarchical coherence: aggregates = sum of components")

## Reconciliation Impact Demonstration

In [ ]:
# Demonstrate the impact of reconciliation on forecast coherence

# Create synthetic incoherent forecasts
n_series = aggregation_matrix.shape[0]
n_bottom = aggregation_matrix.shape[1]

# Generate incoherent base forecasts
np.random.seed(123)
incoherent_forecasts = {}
for i in range(n_series):
    series_id = f"series_{i}"
    incoherent_forecasts[series_id] = np.random.uniform(50, 150, 28)

# Initialize reconciler and fit
reconciler = ProbabilisticReconciler(
    method="probabilistic_mint",
    weights="ols"
)

reconciler.fit(aggregation_matrix)

# Apply reconciliation
reconciled_forecasts, _ = reconciler.reconcile(incoherent_forecasts)

# Compute coherence scores
coherence_before = reconciler.compute_coherence_score(incoherent_forecasts)
coherence_after = reconciler.compute_coherence_score(reconciled_forecasts)

print("Reconciliation Impact:")
print(f"Coherence before reconciliation: {coherence_before:.4f}")
print(f"Coherence after reconciliation:  {coherence_after:.4f}")
print(f"Improvement: {coherence_after - coherence_before:.4f}")

# Visualize the improvement
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Coherence comparison
coherence_data = ['Before Reconciliation', 'After Reconciliation']
coherence_values = [coherence_before, coherence_after]
colors = ['lightcoral', 'lightgreen']

bars = axes[0].bar(coherence_data, coherence_values, color=colors, alpha=0.8)
axes[0].set_title('Hierarchical Coherence Improvement')
axes[0].set_ylabel('Coherence Score (0-1)')
axes[0].set_ylim(0, 1)

# Add value labels
for bar, value in zip(bars, coherence_values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{value:.4f}', ha='center', va='bottom', fontweight='bold')

# Forecast adjustment distribution
adjustments = []
for series_id in incoherent_forecasts.keys():
    if series_id in reconciled_forecasts:
        original = incoherent_forecasts[series_id]
        reconciled = reconciled_forecasts[series_id]
        relative_change = np.mean(np.abs(reconciled - original) / (np.abs(original) + 1e-8))
        adjustments.append(relative_change)

axes[1].hist(adjustments, bins=20, alpha=0.7, color='skyblue', edgecolor='navy')
axes[1].set_title('Distribution of Forecast Adjustments')
axes[1].set_xlabel('Relative Adjustment Size')
axes[1].set_ylabel('Number of Series')
axes[1].axvline(np.mean(adjustments), color='red', linestyle='--', 
                label=f'Mean: {np.mean(adjustments):.3f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nMean relative adjustment: {np.mean(adjustments):.3f}")
print(f"This shows how much forecasts are typically adjusted to maintain coherence.")

## Model Architecture Overview

In [ ]:
# Create a visual representation of the model architecture
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, ConnectionPatch

fig, ax = plt.subplots(1, 1, figsize=(14, 10))
ax.set_xlim(0, 10)
ax.set_ylim(0, 12)
ax.axis('off')

# Define colors
colors = {
    'data': 'lightblue',
    'statistical': 'lightgreen', 
    'deep_learning': 'lightyellow',
    'ensemble': 'lightcoral',
    'reconciliation': 'lightpink',
    'output': 'lightgray'
}

# Helper function to create boxes
def create_box(ax, x, y, width, height, text, color, fontsize=10):
    box = FancyBboxPatch(
        (x, y), width, height,
        boxstyle="round,pad=0.1",
        facecolor=color,
        edgecolor='black',
        linewidth=1
    )
    ax.add_patch(box)
    ax.text(x + width/2, y + height/2, text, ha='center', va='center', 
            fontsize=fontsize, fontweight='bold', wrap=True)
    return box

# Data layer
create_box(ax, 0.5, 10, 9, 1, 'M5 Hierarchical Time Series Data', colors['data'], 12)

# Statistical models
create_box(ax, 0.5, 8, 2, 1, 'ETS\nModel', colors['statistical'])
create_box(ax, 2.8, 8, 2, 1, 'ARIMA\nModel', colors['statistical'])

# Deep learning models
create_box(ax, 5.6, 8, 2, 1, 'Temporal Fusion\nTransformer', colors['deep_learning'])
create_box(ax, 7.9, 8, 2, 1, 'N-BEATS\nModel', colors['deep_learning'])

# Ensemble layer
create_box(ax, 2, 6, 6, 1, 'Weighted Ensemble Combination', colors['ensemble'], 12)

# Reconciliation layer
create_box(ax, 1, 4, 8, 1, 'Probabilistic Hierarchical Reconciliation\n(MinT with Uncertainty Preservation)', 
           colors['reconciliation'], 11)

# Output layer
create_box(ax, 0.5, 2, 4.3, 1, 'Coherent Point\nForecasts', colors['output'])
create_box(ax, 5.2, 2, 4.3, 1, 'Coherent Prediction\nIntervals', colors['output'])

# Add arrows
arrow_props = dict(arrowstyle='->', lw=2, color='black')

# Data to models
ax.annotate('', xy=(1.5, 9), xytext=(2, 10), arrowprops=arrow_props)
ax.annotate('', xy=(3.8, 9), xytext=(3.5, 10), arrowprops=arrow_props)
ax.annotate('', xy=(6.6, 9), xytext=(6.5, 10), arrowprops=arrow_props)
ax.annotate('', xy=(8.9, 9), xytext=(8, 10), arrowprops=arrow_props)

# Models to ensemble
ax.annotate('', xy=(3, 7), xytext=(1.5, 8), arrowprops=arrow_props)
ax.annotate('', xy=(4, 7), xytext=(3.8, 8), arrowprops=arrow_props)
ax.annotate('', xy=(6, 7), xytext=(6.6, 8), arrowprops=arrow_props)
ax.annotate('', xy=(7, 7), xytext=(8.9, 8), arrowprops=arrow_props)

# Ensemble to reconciliation
ax.annotate('', xy=(5, 5), xytext=(5, 6), arrowprops=arrow_props)

# Reconciliation to outputs
ax.annotate('', xy=(2.7, 3), xytext=(4, 4), arrowprops=arrow_props)
ax.annotate('', xy=(7.3, 3), xytext=(6, 4), arrowprops=arrow_props)

# Add title
ax.text(5, 11.5, 'Hierarchical Ensemble Forecasting with Uncertainty Quantification', 
        ha='center', va='center', fontsize=16, fontweight='bold')

# Add legend
legend_elements = [
    mpatches.Patch(color=colors['data'], label='Data Layer'),
    mpatches.Patch(color=colors['statistical'], label='Statistical Models'),
    mpatches.Patch(color=colors['deep_learning'], label='Deep Learning Models'),
    mpatches.Patch(color=colors['ensemble'], label='Ensemble Layer'),
    mpatches.Patch(color=colors['reconciliation'], label='Reconciliation Layer'),
    mpatches.Patch(color=colors['output'], label='Output Layer')
]
ax.legend(handles=legend_elements, loc='lower right', bbox_to_anchor=(1, -0.1))

plt.tight_layout()
plt.show()

print("Model Architecture Features:")
print("1. Multi-model ensemble combining statistical and deep learning approaches")
print("2. Probabilistic reconciliation maintains forecast coherence")
print("3. Uncertainty quantification preserved through all hierarchy levels")
print("4. Scalable to large hierarchical datasets like M5")

## Summary and Key Insights

In [ ]:
# Summary statistics and insights
print("=" * 60)
print("HIERARCHICAL FORECAST RECONCILIATION - KEY INSIGHTS")
print("=" * 60)

print("\n🎯 PROJECT OBJECTIVES:")
print("  ✓ Combine multiple forecasting approaches (statistical + deep learning)")
print("  ✓ Maintain hierarchical coherence across all aggregation levels")
print("  ✓ Preserve uncertainty quantification through reconciliation")
print("  ✓ Achieve target performance metrics (WRMSSE < 0.52)")

print("\n📊 DATA CHARACTERISTICS:")
print(f"  • Total series across hierarchy: {total_series:,}")
print(f"  • Time series length: {len(daily_total)} days")
print(f"  • Hierarchy levels: {len(config['data']['aggregation_levels'])}")
print(f"  • Zero sales rate: {(synthetic_data['sales'] == 0).mean():.1%}")

print("\n🔧 MODEL COMPONENTS:")
print("  Statistical Models:")
print("    - ETS (Exponential Smoothing) with seasonal patterns")
print("    - ARIMA with seasonal components")
print("  Deep Learning Models:")
print("    - Temporal Fusion Transformer (attention-based)")
print("    - N-BEATS (neural basis expansion)")
print("  Reconciliation:")
print("    - Minimum Trace with probabilistic constraints")
print("    - Uncertainty preservation through the hierarchy")

print("\n🎯 TARGET METRICS:")
target_metrics_display = {
    "WRMSSE": "< 0.52 (Weighted Root Mean Squared Scaled Error)",
    "Coverage 90%": "> 0.88 (90% prediction interval coverage)", 
    "Reconciliation Coherence": "> 0.99 (Hierarchical coherence score)",
    "CRPS": "< 0.045 (Continuous Ranked Probability Score)"
}

for metric, description in target_metrics_display.items():
    print(f"  • {metric}: {description}")

print("\n🚀 INNOVATIONS:")
print("  1. Novel probabilistic reconciliation preserving uncertainty")
print("  2. Ensemble approach combining diverse model types")
print("  3. End-to-end pipeline with automated hyperparameter optimization")
print("  4. Comprehensive evaluation framework with multiple metrics")

print("\n📈 BUSINESS IMPACT:")
print("  • Improved demand planning accuracy across all business levels")
print("  • Reliable uncertainty estimates for risk management")
print("  • Scalable framework for large retail hierarchies")
print("  • Coherent forecasts supporting consistent decision-making")

print("\n" + "=" * 60)

## Next Steps

To use this framework with real M5 data:

1. **Download M5 Data**: Get the M5 forecasting competition dataset from Kaggle
2. **Configure Paths**: Update the `data.path` in `configs/default.yaml`
3. **Train Models**: Run `python scripts/train.py --optimize-hyperparameters`
4. **Evaluate Results**: Use `python scripts/evaluate.py --create-plots --save-predictions`
5. **Monitor Training**: View results in MLflow UI with `mlflow ui`

The framework is designed to handle the full M5 dataset with 42,840 time series across 11 hierarchy levels.